In [1]:
from PIL import Image
from pathlib import Path
from os import path
import numpy as np
import cv2
import imutils
from time import sleep


class SlicingTemplate:
    def __init__(self):
        self.image_file = None  # posix path
        self.img = None  # actual image
        self.counter = None
        self.save_dir = None
        
        self.left = 0
        self.x1 = 330
        self.x2 = 470
        self.x3 = 990
        self.right = None  # set in self.load()
        
        self.top = 0
        self.y1 = 110
        self.y2 = 235
        self.y3 = 360
        self.y4 = 510
        self.bottom = None  # set in self.load()
        
    # This method was adapted from the original, just for this notebook
    def single_image_process(self):
        self.img = Image.open(str(self.image_file)) # convert to cv2 later...
        self.right = self.img.size[0]  # image width
        self.bottom = self.img.size[1]  # image height
        self.setup_sections()
        
        # crop out the parts based on the (manually set) template's dimensions
        for section in self.major_sections:
            for dir_name, box in section.items():
                cropped_img = self.img.crop((box["left"], box["top"], box["right"], box["bottom"]))
                file_name = path.join(self.save_dir, dir_name, str(self.counter)+".jpg")
                cropped_img.save(file_name)
        
    def setup_sections(self):
        # these are the box boundaries for each section, manually set
        self.major_sections = [
            {"uppercase": {
                "left": self.left,
                "top": self.top,
                "right": self.right,
                "bottom": self.y1}
            },
            {"lowercase": {
                "left": self.left,
                "top": self.y1,
                "right": self.right,
                "bottom": self.y2}
            },
            {"digits": {
                "left": self.left,
                "top": self.y2,
                "right": self.x2,
                "bottom": self.y3}
            },
            {"punctuation": {
                "left": self.x2,
                "top": self.y2,
                "right": self.right,
                "bottom": self.y3}
            },
            {"sent1": {
                "left": self.left,
                "top": self.y3,
                "right": self.x1,
                "bottom": self.y4}
            },
            {"sent2": {
                "left": self.x2,
                "top": self.y3,
                "right": self.x3,
                "bottom": self.y4}
            },
            {"sent3": {
                "left": self.left,
                "top": self.y4,
                "right": self.x1,
                "bottom": self.bottom}
            },
            {"sent4": {
                "left": self.x2,
                "top": self.y4,
                "right": self.x3,
                "bottom": self.bottom}
            }
        ]





IMG_DIR = Path("Samples")
CROPPED_DIR = Path("Version6Results/cropped")
JPG_DIR = Path("Version6Results/jpg")
SLICE_DIR = Path("Version6Results/slices")

counter = 1
for picture in IMG_DIR.iterdir():
    if counter % 3 == 0:
        print("cool down")
        sleep(120) # chance to cool down during large runs
        
    if picture.suffix in [".jpg", ".jpeg", ".png"]:
        file_name = str(counter)+".jpg"
        file_path = path.join(Path(JPG_DIR, file_name))
        save_name = str(file_path)
        BW = "L"
        new_img = Image.open(str(picture)).convert(BW) #change to cv2 cvtColor, then remove pillow import
        new_img.save(file_path)
        actual_img = Image.open(save_name)
        width = actual_img.size[0]
        height = actual_img.size[1]
        
#Rotate
        # rotate to landscape if needed, don't yet know up or down
        if width < height:
            rotated = actual_img.rotate(90, expand=True)
        else:
            rotated = actual_img
        rotated.save(save_name)

        # load images
        TEMPLATE = "template.jpg"
        template_img = cv2.imread(TEMPLATE, 1) #1 is grayscale enum flag
        target_img = cv2.imread(save_name, 1)

        # cropping box to reduce search area for target template
        left = 8500
        top = 8000
        right = left + 700
        bottom = top + 700

        # get the target image from the template
        # crop_img = img[y:y+h, x:x+w] #opencv's x and y are flipped
        cropped_img = target_img[top:bottom, left:right]
        method = cv2.TM_SQDIFF_NORMED  
        result = cv2.matchTemplate(template_img, cropped_img, method) 

        # minimum squared difference; image similarity score is maxVal
        mn, maxVal, mnLoc, maxLoc = cv2.minMaxLoc(result)  

        # exaggerate the values to make it easier to set a cutoff point
        score = round((maxVal*100)*(maxVal*100))
        
        # 9500 points seems to be a good cutoff
        if score < 9500:
            upright_image = cv2.rotate(target_img, cv2.ROTATE_180)
        else:
            upright_image = target_img
        cv2.imwrite(save_name, upright_image)

#Crop
        # cropping box to reduce search area for target template
        left = 850
        top = 710
        right = 12560
        bottom = 7300

        img = cv2.imread(save_name, 1) #1 is grayscale enum flag        
        img = img[top:bottom, left:right]

#Downscale
        # calculate new size
        SCALE_PERCENT = 10 # percent of original size
        width = int(img.shape[1] * SCALE_PERCENT / 100)
        height = int(img.shape[0] * SCALE_PERCENT / 100)
        new_size = (width, height)

        # resize image
        img = cv2.resize(img, new_size, interpolation = cv2.INTER_AREA)
        cv2.imwrite(save_name, img)


#Threshold
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # redundant?
        gray = cv2.bitwise_not(img) # for the threshold function only

        # threshold the image, setting all foreground pixels to 255 and all background pixels to 0
        thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]


#Deskew
        # grab the (x, y) coordinates of all pixel values that are greater than zero, then use these coordinates to
        # compute a rotated bounding box that contains all coordinates
        coords = np.column_stack(np.where(thresh > 0))
        angle = cv2.minAreaRect(coords)[-1]

        # the `cv2.minAreaRect` function returns values in the range [-90, 0); as the rectangle rotates clockwise 
        # the returned angle trends to 0 -- in this special case we need to add 90 degrees to the angle
        if angle < -45:
            angle = -(90 + angle)

        # otherwise, just take the inverse of the angle to make it positive
        else:
            angle = -angle

        # rotate the image to deskew it
        (h, w) = img.shape[:2]
        center = (w // 2, h // 2)
        M = cv2.getRotationMatrix2D(center, angle, 1.0)
        deskewd = cv2.warpAffine(img, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
        cv2.imwrite(save_name, deskewd)

#Slicing
        # extract major sections from each image
        slicer = SlicingTemplate()
        slicer.image_file = file_path # pass a path object to the class
        slicer.image_file = Path(save_name) # pass a path object to the class
        
        slicer.save_dir = str(SLICE_DIR)
        slicer.counter = counter
        slicer.single_image_process()
        
        print("Sliced: ", file_path)
        counter += 1            

/Users/wandalcooper/Library/Python/3.7/lib/python/site-packages/PIL/Image.py:2800: DecompressionBombWarning: Image size (141812970 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,


Sliced:  Version6Results/jpg/1.jpg
Sliced:  Version6Results/jpg/2.jpg
Sliced:  Version6Results/jpg/3.jpg
Sliced:  Version6Results/jpg/4.jpg
Sliced:  Version6Results/jpg/5.jpg
Sliced:  Version6Results/jpg/6.jpg
Sliced:  Version6Results/jpg/7.jpg
Sliced:  Version6Results/jpg/8.jpg
Sliced:  Version6Results/jpg/9.jpg
Sliced:  Version6Results/jpg/10.jpg
Sliced:  Version6Results/jpg/11.jpg


KeyboardInterrupt: 